# <center> Danny's Diner : The Taste of Success
    
source link: https://8weeksqlchallenge.com/case-study-1/

In [2]:
import pyodbc 

In [20]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-C4OVOKC;'
                      'Database=DannyCaseStudy1;'
                      'Trusted_Connection=yes;')

In [4]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM [dannys_diner].[sale]')

print("Have a look at the data from sales table.\n")

for i in cursor:
    print(i)


Have a look at the data from sales table.

('A', '2021-01-01', 1)
('A', '2021-01-01', 2)
('A', '2021-01-07', 2)
('A', '2021-01-10', 3)
('A', '2021-01-11', 3)
('A', '2021-01-11', 3)
('B', '2021-01-01', 2)
('B', '2021-01-02', 2)
('B', '2021-01-04', 1)
('B', '2021-01-11', 1)
('B', '2021-01-16', 3)
('B', '2021-02-01', 3)
('C', '2021-01-01', 3)
('C', '2021-01-01', 3)
('C', '2021-01-07', 3)


In [7]:
try :
    cur = conn.cursor()
    print("This is the data of Members who have Subscribed to Danny's Diner\n")
    QUERY = 'SELECT * FROM [dannys_diner].[members]'
    cur.execute(QUERY)
    print(cur.fetchall())
except Exception as e:
    cur.close()
    print(e)

This is the data of Members who have Subscribed to Danny's Diner

[('A', '2021-01-07'), ('B', '2021-01-09')]


In [8]:
try :
    cur = conn.cursor()
    print("This is the data of our Products\n")
    QUERY = 'SELECT * FROM [dannys_diner].[menu]'
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
    cur.close()
    print(e)

This is the data of our Products

(1, 'sushi', 10)
(2, 'curry', 15)
(3, 'ramen', 12)


## <center> Case Study Questions

**What is the total amount each customer spent at the restaurant?**

In [40]:
try :
    cur = conn.cursor()
    print("Total Amount Spent is as Follows: \n")
    QUERY = '''
            SELECT s.customer_id, '$ '+ CAST(SUM(m.price) as nvarchar) as [Total Spent]
            FROM dannys_diner.sale s
            JOIN dannys_diner.menu m
                ON m.product_id = s.product_id 
            GROUP BY s.customer_id   
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Total Amount Spent is as Follows: 

('A', '$ 76')
('B', '$ 74')
('C', '$ 36')


**How many days has each customer visited the restaurant?**

In [14]:
try :
    cur = conn.cursor()
    print("Days Visited: \n")
    QUERY = '''
            SELECT customer_id, Cast(COUNT(order_date) as nvarchar) + \' days\' as [Days_Visited]
            FROM dannys_diner.sale
            GROUP BY customer_id 
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Days Visited: 

('A', '6 days')
('B', '6 days')
('C', '3 days')


**What was the first item from the menu purchased by each customer?**

In [39]:
try :
    cur = conn.cursor()
    print("First item purchased : \n")
    QUERY = '''
            WITH FO as
            (
            SELECT customer_id, MIN(order_date) as [First Order Date]
            FROM dannys_diner.sale
            GROUP BY customer_id
            )
            SELECT FO.*,m.product_name
            FROM dannys_diner.sale s
            RIGHT JOIN FO
                ON FO.[First Order Date] = s.order_date AND FO.customer_id = s.customer_id
            JOIN [dannys_diner].[menu] m
                ON s.product_id = m.product_id
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

First item purchased : 

('A', '2021-01-01', 'sushi')
('A', '2021-01-01', 'curry')
('B', '2021-01-01', 'curry')
('C', '2021-01-01', 'ramen')
('C', '2021-01-01', 'ramen')


**What is the most purchased item on the menu and how many times was it purchased by all customers?**

In [42]:
try :
    cur = conn.cursor()
    print("Most purchased item: \n")
    QUERY = '''
            SELECT TOP 1 product_name, CAST (Count(s.product_id) as nvarchar) + ' times' as [Times Purchased]
            FROM [dannys_diner].[sale] s
            JOIN [dannys_diner].[menu] m
                ON m.product_id = s.product_id
            GROUP BY product_name
            ORDER BY Count(s.product_id) DESC
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Most purchased item: 

('ramen', '8 times')


**Which item was the most popular for each customer?**

In [44]:
try :
    cur = conn.cursor()
    print("Most popular item: \n\nCustomer  Product ID")
    QUERY = '''
            WITH MP as
            (
            SELECT customer_id, product_id, Count(product_id) as [Products Purchased], 
            DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY Count(product_id) DESC) as RANKS
            FROM [dannys_diner].[sale] 
            GROUP BY customer_id, product_id
            )
            SELECT customer_id, product_id
            FROM MP
            WHERE RANKS = 1
            Order by customer_id
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Most popular item: 

Customer  Product ID
('A', 3)
('B', 1)
('B', 2)
('B', 3)
('C', 3)


**Which item was purchased first by the customer after they became a member?**

In [38]:
try :
    cur = conn.cursor()
    print("Item purchased after becoming member: \n")
    QUERY = '''
            WITH FPOAM as 
            (
            SELECT m.customer_id, s.product_id, DATEDIFF(Day, m.join_date, s.order_date) as DIFF,
            RANK() OVER(PARTITION BY m.customer_id ORDER BY DATEDIFF(Day, m.join_date, s.order_date)) as [rank_of_orders]
            FROM [dannys_diner].[members] m
            JOIN [dannys_diner].[sale] s 
                ON s.customer_id = m.customer_id AND s.order_date >= m.join_date
            )
            SELECT customer_id, product_name
            FROM FPOAM
            JOIN [dannys_diner].[menu] m
                ON m.product_id = FPOAM.[product_id]
            WHERE [rank_of_orders]= 1
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Item purchased after becoming member: 

('A', 'curry')
('B', 'sushi')


**Which item was purchased just before the customer became a member?**

In [37]:
try :
    cur = conn.cursor()
    print("Item purchased before becoming member: \n")
    QUERY = '''
            WITH BSP as
            (
            SELECT m.customer_id, s.product_id,DATEDIFF(Day, m.join_date, s.order_date) as DIFF,
            RANK() OVER(PARTITION BY m.customer_id ORDER BY DATEDIFF(Day, m.join_date, s.order_date)DESC ) as [rank_of_orders] 
            FROM [dannys_diner].[members] m
            JOIN [dannys_diner].[sale] s 
                ON s.customer_id = m.customer_id AND s.order_date < m.join_date
            )
            SELECT customer_id, product_name
            FROM BSP
            JOIN [dannys_diner].[menu] m
                ON m.product_id = BSP.[product_id]
            WHERE [rank_of_orders]= 1
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Item purchased before becoming member: 

('A', 'sushi')
('A', 'curry')
('B', 'sushi')


**What is the total items and amount spent for each member before they became a member?**

In [27]:
try :
    cur = conn.cursor()
    print("Items purchased and amount spent for each member before they became a member: \n")
    QUERY = '''
            WITH PBS as
            (
            SELECT m.customer_id, s.product_id,m.join_date, s.order_date,
            DATEDIFF(Day, m.join_date, s.order_date) as DIFF,
            RANK() OVER(PARTITION BY m.customer_id ORDER BY DATEDIFF(Day, m.join_date, s.order_date)DESC ) as [rank_of_orders] 
            FROM [dannys_diner].[members] m
            JOIN [dannys_diner].[sale] s 
                ON s.customer_id = m.customer_id AND s.order_date < m.join_date
            )
                SELECT customer_id, '$ ' + Cast(SUM(price) as nvarchar) as [Purchase Before Membership]
            FROM PBS 
            JOIN [dannys_diner].[menu] m 
                ON m.product_id = PBS.product_id
            GROUP BY customer_id
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Items purchased and amount spent for each member before they became a member: 

('A', '$ 25')
('B', '$ 40')


**If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?**

In [31]:
try :
    cur = conn.cursor()
    print("Customer Points: \n")
    QUERY = '''
            SELECT s.customer_id,
            Cast(SUM(CASE WHEN m.product_name = 'sushi' THEN 2*10*m.price
                 ELSE 10*m.price
            END) as nvarchar) + ' Points' AS Points
            FROM [dannys_diner].[menu] m
                JOIN [dannys_diner].[sale] s
            ON m.product_id = s.product_id
            GROUP BY s.customer_id
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)
    
except Exception as e:
#     cur.close()
    print(e)

Customer Points: 

('A', '860 Points')
('B', '940 Points')
('C', '360 Points')


**In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?**

In [35]:
try :
    cur = conn.cursor()
    print("Customer Points: \n")
    QUERY = '''
            SELECT s.customer_id, 
            CAST(SUM(CASE 
                    WHEN DATEDIFF(Day,mm.join_date,s.order_date) <=6 AND DATEDIFF(Day,mm.join_date,s.order_date) >= 0 
                         THEN 2*10*m.price
                    WHEN m.product_id = 1 
                         THEN 2*10*m.price 
                    ELSE 10*m.price
            END) as nvarchar) + ' Points' AS Points
            FROM [dannys_diner].[menu] m
            JOIN [dannys_diner].[sale] s
                ON m.product_id = s.product_id
            JOIN [dannys_diner].[members] mm
                ON mm.customer_id = s.customer_id
            WHERE s.order_date <= '2021-01-30'
            GROUP BY s.customer_id
            '''
    cur.execute(QUERY)
    for i in cur:
        print(i)

except Exception as e:
#     cur.close()
    print(e)

Customer Points: 

('A', '1370 Points')
('B', '820 Points')


## <center> Insights

- Till Date Customer A has spent the most followed by customer B.
- Customer A and B visited us 6 times whereas for customer C, only 3 times
- Ramen is the most popular item out of three available and got sold 8 times out of 15 customer visits which accounts for 53% of whole.
- Ramen seems favourite item for customer A and C whereas B likes all three items equally as per the data.
- Customer A is the first Loyal Customer(Subscribing loyality program) followed by B
- Even though Ramen is popular but before joining Customer loyalty program A ordered sushi and curry and B ordered ‘sushi’.
- Customer C has purchased the lowest out of all three customer and also isn't a member of loyalty program. Restraunt team can request customers for feedback so that they can add more customers to their loyality program.